In [1]:
import json
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.width', 1000)
pd.options.display.float_format = '{:.3f}'.format

In [2]:

with open ('data_collection.json','r') as file:
    data = json.load(file)

df_data = pd.DataFrame(data)

In [3]:
df_data.head(5)

,collection,event_type,order_hash,chain,tokenId,seller,buyer,quantity,payment_quantity,payment_token,decimals,transaction,event_timestamp
0,pudgypenguins,sale,,ethereum,3544,0xbcb5dc467d09d518a0eba0bd968a3ecfb37768c8,0x3bb4fa84b120ac0dbb4a6bb0442fe2c47e324a93,1,10240000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,0x91f0599736c8521db24d745d6a6057a05f7566fc67ff355641fca1658a1dbee3,1720528367
1,pudgypenguins,sale,,ethereum,5116,0x29469395eaf6f95920e59f858042f0e28d98a20b,0x0487839da1bca91ae146117fae465f2342b2826b,1,10230000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,0x9cf1e9e15bb2f865217cdf3ef4657196049498b1205c275225f20d068048cef5,1720508879
2,pudgypenguins,sale,,ethereum,7507,0x2cf30c02097ae99db9f572c2466305f58983d394,0x72795f977a732047635282191fb704fa21968abf,1,10220000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,0x39843ab46c1b8d4a392f8779ad66703711b8da1b332cc90ae3563fd566ea2dfc,1720502963
3,pudgypenguins,sale,,ethereum,752,0xeb83e695adcac2e83f290d2d2815fc58e6491d7a,0x22c41c034cba2a0b71eb5ab560b0cfce3ac0c601,1,10412344800000000000,0x0000000000000000000000000000000000000000,18,0x185f69e7978357e1bcd1bbf8710299a1d667036613d92b16e6d922cc6f0ca484,1720500707
4,pudgypenguins,sale,,ethereum,5116,0x852f432f7a4df936a5be98fbd1f8e6bbbfb6aa46,0x20b638f91bf69b0e8de976c96dcc2289939f7d78,1,10261300000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,0x522e6c3cf082f418b1d4b312c05529b441876f8b79aa81cc284294fe0dca741f,1720498295


In [4]:
df_data.shape

(20000, 13)

tìm dữ liệu thiếu bằng 2 cách
1. isnull()
trả về True nếu null và fasle nếu không null
2. notnull()
ngược lại

In [5]:
# missing value
missing_value = df_data.isnull()
missing_value

,collection,event_type,order_hash,chain,tokenId,seller,buyer,quantity,payment_quantity,payment_token,decimals,transaction,event_timestamp
0,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,False,False,False,False,False,False,False,False,False,False,False,False,False
19996,False,False,False,False,False,False,False,False,False,False,False,False,False
19997,False,False,False,False,False,False,False,False,False,False,False,False,False
19998,False,False,False,False,False,False,False,False,False,False,False,False,False


In [6]:
# đếm số lượng dữ liệu bị thiếu ở từng cột
for column in missing_value.columns.values.tolist():
    print(missing_value[column].value_counts())

collection
False    20000
Name: count, dtype: int64
event_type
False    20000
Name: count, dtype: int64
order_hash
False    20000
Name: count, dtype: int64
chain
False    20000
Name: count, dtype: int64
tokenId
False    20000
Name: count, dtype: int64
seller
False    20000
Name: count, dtype: int64
buyer
False    20000
Name: count, dtype: int64
quantity
False    20000
Name: count, dtype: int64
payment_quantity
False    20000
Name: count, dtype: int64
payment_token
False    20000
Name: count, dtype: int64
decimals
False    20000
Name: count, dtype: int64
transaction
False    20000
Name: count, dtype: int64
event_timestamp
False    20000
Name: count, dtype: int64


Phân tích xu hướng giá: Xem xét các mức giá đã bán của các token để xác định xu hướng giá (tăng hay giảm).
Phân tích thời gian giao dịch: Kiểm tra các khoảng thời gian giữa các giao dịch để xem liệu có xu hướng cụ thể nào về thời điểm mua bán.
Phân tích người mua và người bán: Xem liệu có người mua hoặc người bán nào tham gia nhiều lần không, có thể cho thấy một số người chơi chính trong thị trường này.

Bạn nên xử lý thế nào với dữ liệu bị thiếu? 
    1.Bỏ dữ liệu 
        a.Bỏ hàng đó 
        b. Bỏ cột dữ liệu đó
    3.Thay thế cột 
        a. Thay thế bằng giá trị trung bình
        b. Thay thế bằng giá trị thường xuyên suất hiện
        c. Thay thế nớ dựa trên các chức năng khác

Bạn chỉ nên bỏ toàn bộ cột nếu hầu hết các mục trong cột đều trống. Trong tập dữ liệu, không có cột nào đủ trống để loại bỏ hoàn toàn. Bạn có một số quyền tự do trong việc lựa chọn phương pháp nào để thay thế dữ liệu; tuy nhiên, một số phương pháp có vẻ hợp lý hơn những phương pháp khác

xóa những dòng null
Vì đối với những giao dịch bắt buộc phải có token_id đây là lỗi tỏng quá trình lấy dữ lịu
Những token này không thể thíu và fake dữ liệu vì sẽ làm sai lệch thông tin nhận được nên xóa

In [7]:
# # xóa bỏ những giá trị null
# df_data = df_data.dropna()
# df_data

In [8]:
# xóa 1 cột 
del df_data['order_hash']

Kiểm tra kiểu dữ liệu
dùng dtypes để kiểm tra kiểu dữ liệu
dùng astypes để thay đổi kiểu dữ liệu

In [9]:
# kiểu dữ liệu của các thành phần trong bảng
df_data.dtypes

collection          object
event_type          object
chain               object
tokenId             object
seller              object
buyer               object
quantity             int64
payment_quantity    object
payment_token       object
decimals             int64
transaction         object
event_timestamp      int64
dtype: object

In [10]:
# chuyển giá tiền từ cột payment_quantity thành VND đẻ dể hiểu hơn và đổi tên thành price
# 1ETH = 77.000VND
exchange_rate = 77000

# Chuyển đổi cột 'payment_quantity' thành số thực
df_data['payment_quantity'] = df_data['payment_quantity'].astype(float)

# Tính toán giá trị 'price' bằng VND
df_data['price'] = df_data.apply(lambda row: row['payment_quantity'] / 10**row['decimals'] * exchange_rate, axis=1)

# xóa cột payment_quantity
del df_data['payment_quantity']

df_data.head(5)


,collection,event_type,chain,tokenId,seller,buyer,quantity,payment_token,decimals,transaction,event_timestamp,price
0,pudgypenguins,sale,ethereum,3544,0xbcb5dc467d09d518a0eba0bd968a3ecfb37768c8,0x3bb4fa84b120ac0dbb4a6bb0442fe2c47e324a93,1,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,0x91f0599736c8521db24d745d6a6057a05f7566fc67ff355641fca1658a1dbee3,1720528367,788480.000
1,pudgypenguins,sale,ethereum,5116,0x29469395eaf6f95920e59f858042f0e28d98a20b,0x0487839da1bca91ae146117fae465f2342b2826b,1,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,0x9cf1e9e15bb2f865217cdf3ef4657196049498b1205c275225f20d068048cef5,1720508879,787710.000
2,pudgypenguins,sale,ethereum,7507,0x2cf30c02097ae99db9f572c2466305f58983d394,0x72795f977a732047635282191fb704fa21968abf,1,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,0x39843ab46c1b8d4a392f8779ad66703711b8da1b332cc90ae3563fd566ea2dfc,1720502963,786940.000
3,pudgypenguins,sale,ethereum,752,0xeb83e695adcac2e83f290d2d2815fc58e6491d7a,0x22c41c034cba2a0b71eb5ab560b0cfce3ac0c601,1,0x0000000000000000000000000000000000000000,18,0x185f69e7978357e1bcd1bbf8710299a1d667036613d92b16e6d922cc6f0ca484,1720500707,801750.550
4,pudgypenguins,sale,ethereum,5116,0x852f432f7a4df936a5be98fbd1f8e6bbbfb6aa46,0x20b638f91bf69b0e8de976c96dcc2289939f7d78,1,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,0x522e6c3cf082f418b1d4b312c05529b441876f8b79aa81cc284294fe0dca741f,1720498295,790120.100


In [11]:
# chuyển đổi định dạng cột time
df_data['event_timestamp'] = pd.to_datetime(df_data['event_timestamp'], unit='s')
# Extract date and time
df_data['date'] = df_data['event_timestamp'].dt.date
df_data['time'] = df_data['event_timestamp'].dt.time

# Convert to appropriate data types
df_data['date'] = df_data['date'].astype('datetime64[ns]')

# Drop the original timestamp column if no longer needed
df_data.drop(columns=['event_timestamp'], inplace=True)

df_data

,collection,event_type,chain,tokenId,seller,buyer,quantity,payment_token,decimals,transaction,price,date,time
0,pudgypenguins,sale,ethereum,3544,0xbcb5dc467d09d518a0eba0bd968a3ecfb37768c8,0x3bb4fa84b120ac0dbb4a6bb0442fe2c47e324a93,1,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,0x91f0599736c8521db24d745d6a6057a05f7566fc67ff355641fca1658a1dbee3,788480.000,2024-07-09,12:32:47
1,pudgypenguins,sale,ethereum,5116,0x29469395eaf6f95920e59f858042f0e28d98a20b,0x0487839da1bca91ae146117fae465f2342b2826b,1,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,0x9cf1e9e15bb2f865217cdf3ef4657196049498b1205c275225f20d068048cef5,787710.000,2024-07-09,07:07:59
2,pudgypenguins,sale,ethereum,7507,0x2cf30c02097ae99db9f572c2466305f58983d394,0x72795f977a732047635282191fb704fa21968abf,1,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,0x39843ab46c1b8d4a392f8779ad66703711b8da1b332cc90ae3563fd566ea2dfc,786940.000,2024-07-09,05:29:23
3,pudgypenguins,sale,ethereum,752,0xeb83e695adcac2e83f290d2d2815fc58e6491d7a,0x22c41c034cba2a0b71eb5ab560b0cfce3ac0c601,1,0x0000000000000000000000000000000000000000,18,0x185f69e7978357e1bcd1bbf8710299a1d667036613d92b16e6d922cc6f0ca484,801750.550,2024-07-09,04:51:47
4,pudgypenguins,sale,ethereum,5116,0x852f432f7a4df936a5be98fbd1f8e6bbbfb6aa46,0x20b638f91bf69b0e8de976c96dcc2289939f7d78,1,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,0x522e6c3cf082f418b1d4b312c05529b441876f8b79aa81cc284294fe0dca741f,790120.100,2024-07-09,04:11:35
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,the-origin-pass-official,sale,matic,480,0xe062a9e36fec47978ae07a42e06ee906b00f464c,0x05de309d2435b7600ddb4a8b054457b92b08bcd8,1,0x0000000000000000000000000000000000000000,18,0x9cf3b5832f6ea6557a632592078a9eb4b90b75fd828c74b18d03383841cf44bd,48040300.000,2024-07-04,05:39:55
19996,the-origin-pass-official,sale,matic,978,0xa54c07dc519e56a2745034ba05cca4e74b88f991,0x88681abddb27bceb3a1bb49932cc21102e7aed9f,1,0x0000000000000000000000000000000000000000,18,0xf6e67804ea735c541e098a40a0cf2f3d7c3fe56e95ab862c55de95b843727b82,48125000.000,2024-07-04,05:34:01
19997,the-origin-pass-official,sale,matic,367,0xcb99b5ff7c2fdce62d990627f12370d36b6a7248,0x98f54edd81631db100a239b50a6215b87c59bf5c,1,0x0000000000000000000000000000000000000000,18,0x1c9f22929d042de2fe6bd4dcf6f5dc489e31f9a2633500625c762344a779be59,48125000.000,2024-07-04,05:32:31
19998,the-origin-pass-official,sale,matic,710,0xe853ca29162a31eb4ffec87df34f542633e307ec,0x3ad2acdc708bc91e76ee8762552a24b0e66ca515,1,0x0000000000000000000000000000000000000000,18,0xe3b320c80eb5f5da7c93893b0f5ed1d7b2b5a876c704e2a3f6d136883bd3eb28,48125000.000,2024-07-04,05:30:19


In [12]:
df_data.dtypes

collection               object
event_type               object
chain                    object
tokenId                  object
seller                   object
buyer                    object
quantity                  int64
payment_token            object
decimals                  int64
transaction              object
price                   float64
date             datetime64[ns]
time                     object
dtype: object

In [13]:
# kiểm tra các thời gian phổ biến của từng loại collection
df_data.groupby('date')['date'].count()

date
2023-03-17     32
2023-03-18     25
2023-03-19     28
2023-03-20     22
2023-03-21     14
             ... 
2024-07-05    632
2024-07-06    539
2024-07-07    550
2024-07-08    497
2024-07-09    415
Name: date, Length: 455, dtype: int64

In [14]:
# so sánh các loại collection trong 1 khoảng tg cự thể để có phán đoán chính xác hơn
